In [3]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from konlpy.tag import Okt  
okt = Okt() 
import tensorflow as tf
import numpy as np
from collections import Counter
from konlpy.tag import Komoran
komoran = Komoran()
from wordcloud import WordCloud
import matplotlib.pyplot as plt


import urllib.request
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
import os
import json

In [4]:
import matplotlib.pyplot as plt 
%matplotlib inline
from string import punctuation
import warnings
warnings.filterwarnings('ignore')

In [29]:
from sklearn import datasets 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.naive_bayes import MultinomialNB

### 데이터 확인
* IT, 스포츠, ews정치 등 20개의 카테고리 중 하나로 분류하는 데이터 세트를 이용해 다중분류 실행
* 사이킷런에서 제공하는 문서 데이터 세트

In [41]:
# newsdata=fetch_20newsgroups(subset='train')
# subset 부분에 'all'을 넣으면 전체 데이터, 'train'은 훈련 데이터, 'test'를 넣으면 테스트 데이터를 다운로드 할 수 있다.

In [30]:
news = datasets.fetch_20newsgroups()

In [38]:
print(news['DESCR'])

.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This module contains two loaders. The first one,
:func:`sklearn.datasets.fetch_20newsgroups`,
returns a list of the raw texts that can be fed to text feature
extractors such as :class:`~sklearn.feature_extraction.text.CountVectorizer`
with custom parameters so as to extract feature vectors.
The second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,
returns ready-to-use features, i.e., it is not necessary to use a feature
extractor.

**Data Set Characteristics:**

    =================   ==========
    Classes                     20
    Samples total            18846
    Dimensionality

In [40]:
# 분류 카테고리 확인
news.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [45]:
# 입력 데이터와 타깃 데이터
X, y = news.data, news.target

In [46]:
# 데이터 분리 ( train / test )
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 1)

In [47]:
# 데이터 확인
print(X_train[0], y_train)

From: clay@rsd.dl.nec.com (Clay Finley)
Subject: Re: Carrying Arms
Nntp-Posting-Host: rsd21.rsd.dl.nec.com
Organization: NEC America, Radio Software Dept
Distribution: usa
Lines: 22

|> In article <1993Apr5.220457.6800@spdc.ti.com> dwhite@epcot.spdc.ti.com (Dan White) writes:
|> 
|> >However, haven't we already lost our right to bear arms?
|> 
|> >	It seems that in most states, like Texas, a citizen may own a
|> >gun and carry while at his home or business.  But a citizen is severely
|> >restricted from bearing outside these areas.  Here in Texas you cannot
|> >carry in your car except when "traveling" which is usually defined as
|> >"traveling across a county line."  How did this come about?  Are there
|> >any court rulings on the legality of restricting the carrying of a
|> >weapon outside the home?  
|> 

In Texas, it is legal to carry handguns while "traveling", and also to and from
sporting activities.           ^^^^^^^^

Chapter 46 of the Texas State Penal Code does NOT restrict 

###  TF-IDF 벡터 변환

* How to Use Tfidftransformer & Tfidfvectorizer? <br>
https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.YZd4gt8QuUk

In [48]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB   #다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score     #정확도 계신

In [49]:
cv = CountVectorizer()
X_train_cv = cv.fit_transform(X_train)
print(X_train_cv.shape) # 전체 샘플의 개수와 단어의 수

(8485, 110364)


In [50]:
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(X_train_cv)
print(tfidfv.shape)

(8485, 110364)


### 모델 생성

In [36]:
# 다중 분류 나이브 베이즈 분류 모델 객체
model = MultinomialNB(alpha= 0.01)

### 모델 학습

In [52]:
model.fit(tfidfv, y_train)

MultinomialNB(alpha=0.01)

In [53]:
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

MultinomialNB()

In [55]:
X_test_cv = cv.transform(X_test)
tfidfv_test = tfidf_transformer.transform(X_test_cv)

predicted = model.predict(tfidfv_test)

print("accuracy :", accuracy_score(y_test, predicted))

accuracy : 0.9105691056910569


In [59]:
# 학습 데이터 정확도
model.score(tfidfv, y_train)

0.9974071891573365

In [58]:
# 테스트 데이터 정확도
model.score(tfidfv_test, y_test)

0.9105691056910569